# Preparation

In [ ]:
from voxrow.web.domain.value_objects import Settings

In [ ]:
settings: Settings = Settings()

## Duckdb

In [ ]:
%load_ext magic_duckdb
%dql -t relation
%dql SELECT 'Hanya Tes' ini;

In [ ]:
from os import environ

In [ ]:
environ["AWS_ACCESS_KEY_ID"] = (
    settings.cloudflare_r2.aws_access_key_id.get_secret_value()
)
environ["AWS_SECRET_ACCESS_KEY"] = (
    settings.cloudflare_r2.aws_secret_access_key.get_secret_value()
)

In [ ]:
%%dql -j
CREATE OR REPLACE
SECRET (
    TYPE r2,
    PROVIDER credential_chain,
    CHAIN env,
    ENDPOINT '{{ settings.cloudflare_r2.endpoint_url.host }}'
);

# App

## BPS: Inflation

In [ ]:
!voxrow-web extract-bps-inflation

## IDX: Stock Summary

Oldest date is 2020-01-02

In [ ]:
!voxrow-web extract-idx-stock-summary --start-date 2020-01-01 --end-date 2020-12-31

### Query

In [ ]:
%%dql
SELECT
    *
FROM
    'r2://datalake/idx.co.id/GetStockSummary/2020-01-02.json.gz'
;

In [ ]:
import boto3
import duckdb
from pandas import DataFrame

from voxrow.data.domain.value_objects import Boto3Credential

In [ ]:
credential: Boto3Credential = settings.cloudflare_r2

duckdb.register(
    "list_idx_stock_summary",
    DataFrame(
        content
        for page in (
            boto3.client(
                service_name=credential.service_name,
                endpoint_url=str(credential.endpoint_url),
                aws_access_key_id=credential.aws_access_key_id.get_secret_value(),
                aws_secret_access_key=credential.aws_secret_access_key.get_secret_value(),
                region_name=credential.region_name,
            )
            .get_paginator("list_objects_v2")
            .paginate(
                Bucket="datalake",
                Prefix="idx.co.id/GetStockSummary/",
            )
        )
        for content in page["Contents"]
    ),
)

In [ ]:
%%dql
SELECT
    "date"."year" || '-' || "date"."month" AS "month",
    COUNT(*) AS files
FROM
    list_idx_stock_summary,
    LATERAL (
        SELECT
            REGEXP_EXTRACT(
                "Key",
                '.*/(\d{4}).(\d{2}).\d{2}.json.gz',
                ['year', 'month']
            ) AS "date"
    )
GROUP BY
    1
ORDER BY
    1
;

In [ ]:
%%dql
SELECT
    "date"."year",
    COUNT(
        DISTINCT
        "date"."year" || '-' || "date"."month"
    ) AS months,
    COUNT(*) AS files
FROM
    list_idx_stock_summary,
    LATERAL (
        SELECT
            REGEXP_EXTRACT(
                "Key",
                '.*/(\d{4}).(\d{2}).\d{2}.json.gz',
                ['year', 'month']
            ) AS "date"
    )
GROUP BY
    1
ORDER BY
    1
;